In [1]:
%%html
<script>
(function() {
  // Create the toggle button
  const rtlButton = document.createElement("button");
  rtlButton.textContent = "Toggle LTR";
  rtlButton.id = "top-rtl-toggle";
  rtlButton.style.marginLeft = "8px";
  rtlButton.style.padding = "4px 10px";
  rtlButton.style.fontSize = "14px";
  rtlButton.style.cursor = "pointer";

  // State
  var rtlActive = false;

  // Styling function
  var applyStyleToEditor = (editor) => {
    if (!editor) return;
    var direction = getComputedStyle(editor).getPropertyValue('direction')=='rtl' ? 'ltr' : 'rtl';
    var text_align = getComputedStyle(editor).getPropertyValue('text-align')=='right' ? 'left' : 'right';
    editor.style.setProperty('direction', direction, 'important');
    editor.style.setProperty('text-align', text_align, 'important');
  };

  // Toggle logic
  rtlButton.onclick = () => {
    rtlActive = !rtlActive;
    rtlButton.textContent = rtlActive ? "Toggle LTR" : "Toggle RTL";
    document.querySelectorAll('.jp-MarkdownCell .jp-InputArea-editor').forEach(applyStyleToEditor);
    document.querySelectorAll('.jp-RenderedHTMLCommon code, .jp-RenderedHTMLCommon code span').forEach(applyStyleToEditor);
    document.querySelectorAll('jp-RenderedHTMLCommon, .jp-RenderedHTMLCommon *').forEach(applyStyleToEditor);
  };

  // Watch for focus into editing Markdown cells
  // document.addEventListener('focusin', (event) => {
  //   const editor = event.target.closest('.jp-MarkdownCell .jp-InputArea-editor');
  //    if (editor) applyStyleToEditor(editor);
  // });

  // Insert into top toolbar if not already present
  var insertIntoToolbar = () => {
    const toolbar = document.querySelector('.jp-NotebookPanel-toolbar');
    if (toolbar && !document.getElementById("top-rtl-toggle")) {
      toolbar.appendChild(rtlButton);
    } else {
      // Try again in a moment if toolbar isn't ready yet
      setTimeout(insertIntoToolbar, 300);
    }
  };

  insertIntoToolbar();
})();
</script>

In [2]:
%%html
<!-- <style>
  table {display: inline-block}
</style> -->

# קבועים פיזיקליים
**SciPy** מוסיפה לסביבה של NumPy כלים נומריים מתקדמים: אינטגרציה, פתרון ODEs, אופטימיזציה, אינטרפולציה, FFT, עיבוד אותות, אלגברה ליניארית, סטטיסטיקה ועוד.  
**קבועים פיזיקליים** זמינים ב־`scipy.constants`: למשל $c, h, k_B, G$ וכד'.

In [3]:
# %%
# Constants and quick NumPy recap — EXPANDED
# (Safe to run even if previous cells didn't import constants)
import numpy as np
from scipy import constants
from scipy.constants import physical_constants  # for named constants like Wien's b

# --- Core physical constants (SI units) ---
c   = constants.c               # speed of light (m/s)
h   = constants.h               # Planck constant (J·s)
hbar= constants.hbar            # reduced Planck constant (J·s)
kB  = constants.Boltzmann       # Boltzmann constant (J/K)
e   = constants.e               # elementary charge (C)
eV  = constants.electron_volt   # 1 eV in Joule (J)
NA  = constants.N_A             # Avogadro number (1/mol)
R   = constants.R               # molar gas constant (J/(mol·K))
sigma = constants.Stefan_Boltzmann  # Stefan–Boltzmann constant (W/(m^2·K^4))
epsilon0 = constants.epsilon_0  # vacuum permittivity (F/m)
mu0     = constants.mu_0        # vacuum permeability (N/A^2)
G   = constants.G               # gravitational constant (m^3/(kg·s^2))
g0  = constants.g               # standard gravity (m/s^2)

# Wien's wavelength-displacement constant b ≈ 2.897771955e-3 m·K (via CODATA)
b_wien = physical_constants["Wien wavelength displacement law constant"][0]

# --- Pretty printing helper (engineering/scientific) ---
def fmt(val, unit):
    """Format a number in scientific notation with a unit."""
    return f"{val:.6e} {unit}"

print("Some constants:")
print("  c       =", fmt(c, "m/s"))
print("  h       =", fmt(h, "J·s"))
print("  ħ       =", fmt(hbar, "J·s"))
print("  k_B     =", fmt(kB, "J/K"))
print("  e       =", fmt(e, "C"))
print("  1 eV    =", fmt(eV, "J"))
print("  N_A     =", fmt(NA, "1/mol"))
print("  R       =", fmt(R, "J/(mol·K)"))
print("  σ       =", fmt(sigma, "W/(m^2·K^4)"))
print("  ε0      =", fmt(epsilon0, "F/m"))
print("  μ0      =", fmt(mu0, "N/A^2"))
print("  G       =", fmt(G, "m^3/(kg·s^2)"))
print("  g0      =", fmt(g0, "m/s^2"))
print("  b_Wien  =", fmt(b_wien, "m·K"))


Some constants:
  c       = 2.997925e+08 m/s
  h       = 6.626070e-34 J·s
  ħ       = 1.054572e-34 J·s
  k_B     = 1.380649e-23 J/K
  e       = 1.602177e-19 C
  1 eV    = 1.602177e-19 J
  N_A     = 6.022141e+23 1/mol
  R       = 8.314463e+00 J/(mol·K)
  σ       = 5.670374e-08 W/(m^2·K^4)
  ε0      = 8.854188e-12 F/m
  μ0      = 1.256637e-06 N/A^2
  G       = 6.674300e-11 m^3/(kg·s^2)
  g0      = 9.806650e+00 m/s^2
  b_Wien  = 2.897772e-03 m·K


נדגים שימוש באמצעות מספר דוגמאות broadcasting ווקטוריזציה:

####  1) חישוב אנרגיית פוטון מאורך גל
האור הוא קרינה אלקטרומגנטית, ובתיאור הקוונטי שלו הוא מורכב מחלקיקים הנקראים **פוטונים**. לכל פוטון אנרגיה בדידה, הפרופורציונית לתדירות הגל: $E = h f$ (כאשר $h$ הוא קבוע פלאנק). מאחר שמהירות האור נקבעת על־ידי $c = \lambda f$, מתקבל הקשר השימושי  

$E = \dfrac{h c}{\lambda}$.  

אורך גל קצר יותר (כחול/על־סגול) משמעו אנרגיה גדולה יותר, ואורך גל ארוך יותר (אדום/תת־אדום) משמעו אנרגיה קטנה יותר — עיקרון שמסביר תופעות כמו בליעה/פליטה ספקטרלית, יעילות תאים סולאריים ורגישות הראייה.

בדוגמה יצרנו וקטור של אורכי גל אופייניים בתחום הנראה (ננומטר), המרנו למטרים, וחישבנו את $E$ לכל ערך — בבת אחת — באמצעות וקטוריזציה של NumPy (כלומר בלי לולאות מפורשות). לבסוף ביצענו המרות יחידות: מג'ול ל־eV בעזרת $1\ \mathrm{eV}$ במונחי ג'ול מתוך `scipy.constants`. כך מקבלים אינטואיציה: ככל שאורך הגל קצר יותר ($\lambda$ קטן), האנרגיה גדולה יותר.


In [4]:
# 1) Photon energy from wavelength: E = h*c/λ (vectorized over λ)
wavelength_nm = np.array([400, 500, 600, 700])           # nm
wavelength_m  = wavelength_nm * 1e-9                      # convert to meters
E_J = h * c / wavelength_m                                # Joule
E_eV = E_J / eV                                           # electron-volt
print("\nPhoton energy for visible wavelengths:")
for lam, Ej, Ev in zip(wavelength_nm, E_J, E_eV):
    print(f"  λ = {lam:>3.0f} nm  ->  E = {Ej:8.3e} J  = {Ev:7.3f} eV")


Photon energy for visible wavelengths:
  λ = 400 nm  ->  E = 4.966e-19 J  =   3.100 eV
  λ = 500 nm  ->  E = 3.973e-19 J  =   2.480 eV
  λ = 600 nm  ->  E = 3.311e-19 J  =   2.066 eV
  λ = 700 nm  ->  E = 2.838e-19 J  =   1.771 eV


#### 2) סקאלת אנרגיה תרמית $k_B T$ ומתיחות תרמית במתח
במערכות פיזיקליות בטמפרטורה סופית, החלקיקים אינם נחים אלא נעים באנרגיות אקראיות הנובעות מחום. קנה־המידה הטבעי לאנרגיה תרמית הוא $k_B T$, כאשר $k_B$ הוא קבוע בולצמן ו־$T$ היא הטמפרטורה בקלווין. גודל זה מייצג את האנרגיה הממוצעת של תנודות תרמיות, ומשמש בהבנת תופעות כמו הולכה חשמלית, דיפוזיה ותהליכים סטטיסטיים.  
באלקטרוניקה מקובל להמיר את האנרגיה התרמית למתח אופייני באמצעות $V_T = \tfrac{k_B T}{e}$, כאשר $e$ הוא מטען האלקטרון. מתח זה מהווה קנה־מידה חשוב לקביעת התנהגות של דיודות, טרנזיסטורים ורכיבים מוליכים למחצה, ובטמפרטורת חדר ערכו קרוב ל־$25\ \mathrm{mV}$.


בדוגמה זו חישבנו עבור מספר טמפרטורות אופייניות את $k_B T$ — סדר־גודל האנרגיות החום־תרמיות במערכת. המרה ל־eV מקלה על ההשוואה לפיזיקה של אלקטרונים ומוליכים למחצה. בנוסף חישבנו את ה״מתח התרמי״ $V_T = \dfrac{k_B T}{e}$, קנה־מידה חשוב באלקטרוניקה (כ־$25$–$26\ \mathrm{mV}$ בטמפרטורת חדר). שוב, הכל נעשה על מערך של $T$ים בבת אחת (וקטוריזציה).



In [5]:
# 2) Thermal energy scale: k_B T and thermal voltage (k_B T / e) at a few temperatures
T = np.array([77, 300, 600, 1000])                        # K (LN2, room, hot, very hot)
kT_J = kB * T
kT_eV = kT_J / eV
thermal_voltage = kT_J / e                                # Volts
print("\nThermal scales (vectorized over T):")
for Ti, kTJ, kTe, Vt in zip(T, kT_J, kT_eV, thermal_voltage):
    print(f"  T = {Ti:>4.0f} K  ->  k_B T = {kTJ:.3e} J = {kTe:.4f} eV,  k_B T / e = {Vt*1e3:.2f} mV")


Thermal scales (vectorized over T):
  T =   77 K  ->  k_B T = 1.063e-21 J = 0.0066 eV,  k_B T / e = 6.64 mV
  T =  300 K  ->  k_B T = 4.142e-21 J = 0.0259 eV,  k_B T / e = 25.85 mV
  T =  600 K  ->  k_B T = 8.284e-21 J = 0.0517 eV,  k_B T / e = 51.70 mV
  T = 1000 K  ->  k_B T = 1.381e-20 J = 0.0862 eV,  k_B T / e = 86.17 mV


#### 3) דוגמת Broadcasting: אנרגיה פוטנציאלית כבידתית
במערכת מכנית פשוטה, אנרגיה פוטנציאלית נגרמת מכוח הכבידה הפועל על מסה $m$ בגובה $h$ מעל נקודת ייחוס: $U = m g h$, כאשר $g$ הוא תאוצת הכובד (כ־$9.8\ \mathrm{m/s^2}$ על פני כדור הארץ). קשר זה חשוב להבנת תהליכים יומיומיים כמו נפילת עצמים, עבודה מכנית והמרת אנרגיה במערכות פיזיקליות. במודלים מתקדמים יותר, עקרון זה מהווה בסיס לחישוב שדות כבידה ולחוקי שימור אנרגיה.

רצינו את האנרגיה $U = m g h$ לכל צירוף של כמה מסות וכמה גבהים. שימוש ב־**Broadcasting** מאפשר לחשב מטריצה $U$ במכה אחת: ממד המסות כעמודות והגבהים כשורות (או להפך) באמצעות הוספת מימד חדש (`masses[:, None]` ו־`heights[None, :]`). זו פרקטיקה חשובה לעבודה יעילה עם טבלאות/סריגים בלי לולאות.


In [6]:
# 3) Broadcasting example: Gravitational potential energy U = m * g0 * h
#    masses shape (3,), heights shape (4,) -> U shape (3,4) via broadcasting
masses = np.array([0.2, 1.0, 5.0])                        # kg
heights = np.array([0.5, 1.0, 2.0, 10.0])                 # m
U = masses[:, None] * g0 * heights[None, :]               # J, shape (3,4)
print("\nGravitational potential energy matrix U[mass, height] = m * g0 * h (J):")
print(U)


Gravitational potential energy matrix U[mass, height] = m * g0 * h (J):
[[  0.980665   1.96133    3.92266   19.6133  ]
 [  4.903325   9.80665   19.6133    98.0665  ]
 [ 24.516625  49.03325   98.0665   490.3325  ]]


#### 4) חוק וין: אורך גל שיא של גוף שחור
בפיזיקה של תרמודינמיקה וקרינה, **גוף שחור** הוא מודל אידיאלי שקולט את כל הקרינה הפוגעת בו ופולט קרינה אלקטרומגנטית בהתאם לטמפרטורתו בלבד. ספקטרום הפליטה מתואר על־ידי חוק פלאנק, אך מיקום השיא בספקטרום נקבע על־פי **חוק וין להזחת השיא**:  
$\lambda_{\max} = \dfrac{b}{T}$,  
כאשר $b$ הוא קבוע וין ו־$T$ היא הטמפרטורה בקלווין. החוק מראה שככל שהטמפרטורה גבוהה יותר, אורך הגל של השיא קטן יותר — ולכן עצמים חמים פולטים קרינה בתחומי אור קצרים (כחול/על־סגול), בעוד שעצמים קרים יותר פולטים בעיקר באורכי גל ארוכים (אינפרה־אדום). עיקרון זה מסביר את צבעי הכוכבים ואת שיטות למדידת טמפרטורה באמצעות קרינה.

חוק וין קובע $\lambda_{\max} = \dfrac{b}{T}$ עם קבוע $b$ שנשלף מ־`physical_constants`. חישבנו את $\lambda_{\max}$ עבור כמה טמפרטורות (חדר, נורת להט, שמש) והמרנו לננומטר לקבלת תחושת סדרי גודל. הדוגמה מראה כיצד לשלב קבועים סטנדרטיים עם חישובים וקטוריים — ובמקביל לחזק אינטואיציה: ככל שהטמפרטורה גבוהה יותר, שיא הקרינה מוסת לכיוון אורכי גל קצרים יותר.



In [7]:
# 4) Blackbody peak wavelength via Wien's law: λ_max = b / T (vectorized)
T_bb = np.array([300, 1000, 3000, 5800])                  # K (room, hot, incandescent-ish, Sun ~5800 K)
lambda_max_m = b_wien / T_bb
lambda_max_nm = lambda_max_m * 1e9
print("\nBlackbody peak wavelength (Wien's law):")
for Ti, lam_m, lam_nm in zip(T_bb, lambda_max_m, lambda_max_nm):
    print(f"  T = {Ti:>4.0f} K  ->  λ_max ≈ {lam_m:.3e} m  (~{lam_nm:.0f} nm)")


Blackbody peak wavelength (Wien's law):
  T =  300 K  ->  λ_max ≈ 9.659e-06 m  (~9659 nm)
  T = 1000 K  ->  λ_max ≈ 2.898e-06 m  (~2898 nm)
  T = 3000 K  ->  λ_max ≈ 9.659e-07 m  (~966 nm)
  T = 5800 K  ->  λ_max ≈ 4.996e-07 m  (~500 nm)


### 5) הקדמה קצרה: מהירות האור וקבועי היסוד באלקטרומגנטיות  
באלקטרודינמיקה קלאסית (EMT), מהירות התפשטות של גלי אלקטרומגנטיים בוואקום אינה רק גודל ניסיוני אלא נובעת ישירות מקבועי היסוד:  
$c = \dfrac{1}{\sqrt{\varepsilon_0 \mu_0}}$,  
כאשר $\varepsilon_0$ הוא המקדם הדיאלקטרי של הריק (permitivity) ו־$\mu_0$ הוא המקדם המגנטי של הריק (permeability). קשר זה מדגים כיצד תכונות הוואקום מכתיבות את מהירות האור, ומהווה דוגמה מרתקת לאחדות של חשמל, מגנטיות וקרינה. מעבר לערך עצמו, בדיקה כזו חשובה לאימות עקביות יחידות וחישובים כשעובדים עם קבועים מספריים ממקורות שונים.

ב־EMT ידוע שבוואקום $c = \dfrac{1}{\sqrt{\varepsilon_0 \mu_0}}$. בדוגמה חישבנו את הצד הימני מקבועים פיזיקליים והשווינו למהירות האור `constants.c`. חישוב **שגיאה יחסית** קטן מאשש עקביות יחידות וערכים, ומהווה בדיקת־שפיות שימושית כשעובדים עם קבועים ממקורות שונים.


In [8]:
# 5) Quick sanity check: c ≈ 1/sqrt(ε0 * μ0)
c_from_eps_mu = 1.0 / np.sqrt(epsilon0 * mu0)
rel_err = abs(c_from_eps_mu - c) / c
print("\nSanity check: c vs 1/sqrt(ε0 μ0)")
print(f"  c_from_eps_mu = {c_from_eps_mu:.6e} m/s")
print(f"  relative error = {rel_err:.3e}")


Sanity check: c vs 1/sqrt(ε0 μ0)
  c_from_eps_mu = 2.997925e+08 m/s
  relative error = 5.967e-13


In [2]:
import json
from jupyterquiz import display_quiz

quiz_json = \
'''
[{
  "question": "הקוד הבא עושה שימוש בקבועים מתוך scipy.constants:<br><br>from scipy import constants<br>c = constants.c  # מהירות האור (m/s)<br>h = constants.h  # קבוע פלאנק (J·s)<br>kB = constants.Boltzmann  # קבוע בולצמן (J/K)<br><br>מהי התכונה המרכזית של הקבועים הללו וכיצד נשתמש בהם נכון?",
  "type": "many_choice",
  "answers": [
    {
      "answer": "הקבועים מוגדרים ביחידות SI מדויקות לפי CODATA וניתנים לשימוש ישיר בחישובים פיזיקליים",
      "correct": true,
      "feedback": "נכון! scipy.constants מספק ערכים עדכניים ביחידות SI תקניות לשימוש מיידי."
    },
    {
      "answer": "הקבועים ניתנים רק ביחידות לא־SI ויש להמיר אותם תמיד באופן ידני",
      "correct": false,
      "feedback": "לא נכון — כברירת מחדל הקבועים ניתנים ביחידות SI. המרה נדרשת רק אם רוצים יחידות אחרות."
    },
    {
      "answer": "הקבועים משתנים בין ריצות ולכן אינם מתאימים לחישובים מדויקים",
      "correct": false,
      "feedback": "ממש לא — הערכים קבועים ואחידים לפי ההגדרות הפיזיקליות המקובלות."
    },
    {
      "answer": "הקבועים זמינים ב-numpy אך לא ב-scipy",
      "correct": false,
      "feedback": "לא נכון — numpy לא כולל קבועים פיזיקליים; רק scipy.constants מספק אותם."
    }
  ]
}]
'''
myquiz = json.loads(quiz_json)
display_quiz(myquiz)


ModuleNotFoundError: No module named 'jupyterquiz'